<a href="https://colab.research.google.com/github/margaridagomes/dataeng-basic-course/blob/main/spark_streaming/examples/example_2_rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Usecase 2
- Reading data from "rate"
- Aggregating data by window time
- Checking results from query in memory

# Setting up PySpark

In [ ]:
%pip install pyspark

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('Test streaming').getOrCreate()

# Write output in memory

In [2]:
import pyspark.sql.functions as F

# read stream
stream1 = spark.readStream.format("rate").option("rowsPerSecond", 10).load()

# transform
transformed = stream1.withColumn("minute", F.minute("timestamp"))
agg = transformed.groupBy(F.window(transformed.timestamp, "5 seconds")).count()

# write stream in memory
query = (agg.writeStream
.format('memory')
.queryName('my_query')
.outputMode('complete')
.start()
#.awaitTermination(10) #definir um tempo para esperar o processamento, neste caso 10s, o streaming fica bloqueado
)

In [3]:
query.isActive

True

In [44]:
spark.sql("select * from my_query order by window desc").show(10,False)

+------------------------------------------+-----+
|window                                    |count|
+------------------------------------------+-----+
|{2025-07-11 21:46:25, 2025-07-11 21:46:30}|17   |
|{2025-07-11 21:46:20, 2025-07-11 21:46:25}|50   |
|{2025-07-11 21:46:15, 2025-07-11 21:46:20}|50   |
|{2025-07-11 21:46:10, 2025-07-11 21:46:15}|50   |
|{2025-07-11 21:46:05, 2025-07-11 21:46:10}|50   |
|{2025-07-11 21:46:00, 2025-07-11 21:46:05}|33   |
+------------------------------------------+-----+



In [45]:
query.stop()

# Write output as json

In [46]:
!rm -rf content/output

In [47]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame

def save_parquet(df, batch_id):
  (df
   .withColumn("batch_id",F.lit(batch_id))
   .withColumn("load_time",F.current_timestamp())
   .write.mode("append")
   .parquet("content/output/rate_parquet")
  )

# read stream
stream1 = spark.readStream.format("rate").option("rowsPerSecond", 10).load()

# transform
transformed = stream1.withWatermark("timestamp", "5 seconds").withColumn("minute", F.minute("timestamp"))
agg = transformed.groupBy(F.window(transformed.timestamp, "5 seconds")).count()

# write stream as parquet with foreachBatch
query = (agg.writeStream
.option('checkpointLocation', 'content/output/checkpoint')
.trigger(processingTime='20 seconds')
.outputMode('append')
.foreachBatch(save_parquet) #quando queremos escrever output, ou código customizado p.e. escrever em duas BDs diferentes; escreve streaming em formato batch;
.start()
)


In [48]:
result = spark.read.format("parquet").load("content/output/rate_parquet/")
result.sort(F.asc("window")).show(100, False)

+------------------------------------------+-----+--------+--------------------------+
|window                                    |count|batch_id|load_time                 |
+------------------------------------------+-----+--------+--------------------------+
|{2025-07-11 21:47:15, 2025-07-11 21:47:20}|28   |2       |2025-07-11 21:47:44.289153|
|{2025-07-11 21:47:20, 2025-07-11 21:47:25}|50   |2       |2025-07-11 21:47:44.289153|
+------------------------------------------+-----+--------+--------------------------+



In [50]:
query.stop()

In [51]:
query.isActive

False

# Enrich data with faker

In [52]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 40.7 MB/s eta 0:00:00


In [53]:
!rm -rf content/output/events

In [54]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from faker import Faker

def insert_into_table(df, batch_id):
  fake = Faker()
  new_columns = {
      'name': F.lit(fake.name()),
      'address': F.lit(fake.address()),
      'email': F.lit(fake.email()),
      'dob': F.lit(fake.date_of_birth()),
      'phone': F.lit(fake.phone_number())
  }
  df = df.withColumns(new_columns)
  df.write.mode("append").format("parquet").save("content/output/events")

# read stream
df_stream = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# write stream
query = (df_stream.writeStream
.outputMode('append')
.trigger(processingTime='1 seconds')
.foreachBatch(insert_into_table)
.start()
)

In [55]:
query.stop()


In [56]:
spark.read.parquet("content/output/events").show(100, False)

+---------------------+-----+------------------+------------------------------------------------------+-------------------------+----------+--------------------+
|timestamp            |value|name              |address                                               |email                    |dob       |phone               |
+---------------------+-----+------------------+------------------------------------------------------+-------------------------+----------+--------------------+
|2025-07-11 21:48:48.5|6    |Kristin Powell    |9275 Sanders Mews Apt. 253\nFrederickville, UT 46901  |veronicaroth@example.com |1965-03-05|(279)694-1980x43964 |
|2025-07-11 21:48:49.5|7    |Amanda Strong     |664 Tonya Course Suite 549\nPort Melindastad, IA 61067|lhernandez@example.com   |2007-11-10|(946)885-7568x152   |
|2025-07-11 21:48:51.5|9    |Jonathan Patton   |6289 Lisa Trail Apt. 750\nMurrayville, MA 55976       |williamward@example.net  |1922-12-03|001-852-223-1969x180|
|2025-07-11 21:48:42.5|0    

In [57]:
query.stop()